In [1]:
import numpy as np

#localSearch tries to find the best schedule
#data['f'] - some function f(forF) which finds a value for a provided CORRECT schedule. This value should be minimized by localSearch
#data['forF'] - some information which is requered to find the value by f() function
#data['classes'] - the number of classes
#data['days'] - the number of days
#data['hours'] - the number of hours
#data['doNotOverlap'] - a matrix which has [x][y]=1 if the class number x should not be at the same time with y
#data['forbidTime'] - a tensor which has [day][hour][class]=1 if the class can not be put on this hour of this day
#maxiter - how much iteration the main loop of the localSearch can conduct
#neighbours - how much neighours are checked during one loop
#annealing - decreasing temperature with 1 meaning the maximum random and 0 meaning the absense of random
def localSearch(data,maxiter=100,neighbours=10,annealing=[]):
    if len(annealing)==0:
        annealing=np.zeros(maxiter)
    else:
        maxiter=len(annealing)
    c=data['classes']
    d=data['days']
    h=data['hours']
    schedule=np.zeros((d,h,c))
    data['forF']['schedule']=schedule
    ones=[]
    pointers=np.zeros((d,h,c))
    #random distribution
    again=1
    while(again):
        again=0
        for i in range(c):
            if (again==0):
                for j in range(data['requiredHours'][i]):
                    #find just random position
                    maxAttempts=2000
                    iters=0
                    while (iters<maxAttempts):
                        iters+=1
                        chooseD=np.random.randint(d)
                        chooseH=np.random.randint(h)
                        if schedule[chooseD][chooseH][i]==0 and data['forbidTime'][chooseD][chooseH][i]==0:
                            ok=1
                            for k in range(c):
                                if schedule[chooseD][chooseH][k]==1 and (data['doNotOverlap'][i][k]==1 or data['doNotOverlap'][k][i]==1):
                                    ok=0
                            if ok==1:
                                break
                    if iters==maxAttempts:
                        again=1
                        schedule=np.zeros((d,h,c))
                        data['forF']['schedule']=schedule
                        ones=[]
                        pointers=np.zeros((d,h,c))
                        break
                    else:
                        schedule[chooseD][chooseH][i]=1
                        pointers[chooseD][chooseH][i]=len(ones)
                        ones.append((chooseD,chooseH,i))
    #now random distribution is done
    for i in range(maxiter):
        bestValue=data['f'](data['forF'])
        bestSwap=(-1,-1,-1)
        for j in range(neighbours):
            chooseOne=np.random.randint(len(ones))
            iters=0
            maxAttempts=3
            while (iters<maxAttempts):
                iters+=1
                chooseD=np.random.randint(d)
                chooseH=np.random.randint(h)
                if schedule[chooseD][chooseH][ones[chooseOne][2]]==0 and data['forbidTime'][chooseD][chooseH][ones[chooseOne][2]]==0:
                    ok=1
                    for k in range(c):
                        if schedule[chooseD][chooseH][k]==1 and (data['doNotOverlap'][ones[chooseOne][2]][k]==1 or data['doNotOverlap'][k][ones[chooseOne][2]]==1):
                            ok-=1
                            overlap=k
                    if ok>=0:
                        break
            if (iters<maxAttempts):
                if ok==1:
                #try this swap
                    schedule[chooseD][chooseH][ones[chooseOne][2]]=1
                    schedule[ones[chooseOne][0]][ones[chooseOne][1]][ones[chooseOne][2]]=0
                    nowValue=data['f'](data['forF'])
                    #rollback
                    schedule[chooseD][chooseH][ones[chooseOne][2]]=0
                    schedule[ones[chooseOne][0]][ones[chooseOne][1]][ones[chooseOne][2]]=1
                    if nowValue<bestValue:
                        bestSwap=(chooseOne,chooseD,chooseH)
                        bestValue=nowValue
                        bestOverlap=-1
                    #if we have high temperature, we can randomly go there
                    if np.random.rand()<annealing[i]:
                        bestSwap=(chooseOne,chooseD,chooseH)
                        bestValue=nowValue
                        bestOverlap=-1
                        break
                if ok==0:
                #try this swap
                    schedule[chooseD][chooseH][ones[chooseOne][2]]=1
                    schedule[ones[chooseOne][0]][ones[chooseOne][1]][ones[chooseOne][2]]=0
                    schedule[chooseD][chooseH][overlap]=0
                    schedule[ones[chooseOne][0]][ones[chooseOne][1]][overlap]=1
                    nowValue=data['f'](data['forF'])
                    #rollback
                    schedule[chooseD][chooseH][ones[chooseOne][2]]=0
                    schedule[ones[chooseOne][0]][ones[chooseOne][1]][ones[chooseOne][2]]=1
                    schedule[chooseD][chooseH][overlap]=1
                    schedule[ones[chooseOne][0]][ones[chooseOne][1]][overlap]=0
                    if nowValue<bestValue:
                        bestSwap=(chooseOne,chooseD,chooseH)
                        bestValue=nowValue
                        bestOverlap=overlap
                    #if we have high temperature, we can randomly go there
                    if np.random.rand()<annealing[i]:
                        bestSwap=(chooseOne,chooseD,chooseH)
                        bestValue=nowValue
                        bestOverlap=overlap
                        break    
        #now conduct the best swap
        if (bestSwap[0]!=-1):
            if bestOverlap==-1:
                schedule[ones[bestSwap[0]][0]][ones[bestSwap[0]][1]][ones[bestSwap[0]][2]]=0
                pointers[ones[bestSwap[0]][0]][ones[bestSwap[0]][1]][ones[bestSwap[0]][2]]=-1
                schedule[bestSwap[1]][bestSwap[2]][ones[bestSwap[0]][2]]=1
                ones[bestSwap[0]]=(bestSwap[1],bestSwap[2],ones[bestSwap[0]][2])
                pointers[bestSwap[1],bestSwap[2],ones[bestSwap[0]][2]]=bestSwap[0]
            else:
                schedule[ones[bestSwap[0]][0]][ones[bestSwap[0]][1]][ones[bestSwap[0]][2]]=0
                schedule[ones[bestSwap[0]][0]][ones[bestSwap[0]][1]][bestOverlap]=1
                pointers[ones[bestSwap[0]][0]][ones[bestSwap[0]][1]][ones[bestSwap[0]][2]]=-1
                pointers[ones[bestSwap[0]][0]][ones[bestSwap[0]][1]][bestOverlap]=pointers[bestSwap[1]][bestSwap[2]][bestOverlap]
                schedule[bestSwap[1]][bestSwap[2]][ones[bestSwap[0]][2]]=1
                schedule[bestSwap[1]][bestSwap[2]][bestOverlap]=0
                ones[(pointers[bestSwap[1]][bestSwap[2]][bestOverlap]).astype(np.int32)]=(ones[bestSwap[0]][0],ones[bestSwap[0]][1],bestOverlap)
                pointers[bestSwap[1]][bestSwap[2]][bestOverlap]=-1
                ones[bestSwap[0]]=(bestSwap[1],bestSwap[2],ones[bestSwap[0]][2])
                pointers[bestSwap[1],bestSwap[2],ones[bestSwap[0]][2]]=bestSwap[0]
    return (data['f'](data['forF']), schedule)

In [11]:
#an example of using of this local search
def value1(data):#find sum of squares
    res=np.power(data['schedule'].sum(axis=2),2).sum(axis=1).sum(axis=0)
    return res

def value2(data):
    #--bad points for windows for each track - very very bad
    #--bad points for different time of the same class - bad
    #--bad points for putting classes on place which professors don't want - very bad
    #--bad points for a lot of classes at the same time - very very bad if more than 7? at once
    #--bad points if classes are together - very bad
    #--bad points if classes of the same track intersect - bad
    c=data['classes']
    d=data['days']
    h=data['hours']
    schedule=data['schedule']
    dontWant=data['dontWant']
    want=data['want']
    tracks=data['tracks']
    doNotOverlap=data['doNotOverlap']
    res=0
    for i in range(d):
        for j in range(h):
            summ=0
            for k in range(c):
                summ+=schedule[i][j][k]
            if summ>8:
                res+=1000
    for i in range(d):
        for k in range(c):
            if schedule[i][0][k]+schedule[i][1][k]+schedule[i][2][k]>1:
                res+=100
    for i in range(d):
        for k1 in range(c):
            for k2 in range(c):
                if schedule[i][0][k1]==1 and schedule[i][2][k2]==1 and doNotOverlap[k1][k2]==1:
                    res+=2000;
#         tracks1={'E&I':0,'Energy':0,'IT':0,'Biomed':0,'Space':0,'Space/IT/PDAM':0}
#         tracks2={'E&I':0,'Energy':0,'IT':0,'Biomed':0,'Space':0,'Space/IT/PDAM':0}
#         for k in range(c):
#             if schedule[i][0][k]==1:
#                 tracks1[tracks[k]]=1
#         for k in range(c):
#             if schedule[i][1][k]==1:
#                 tracks2[tracks[k]]=1
#         for k in range(c):
#             if schedule[i][2][k]==1:
#                 if tracks1[tracks[k]]==1 and schedule[i][1][k]==0:
#                     res+=10000
    for k in range(c):
        was=np.zeros(h)
        for i in range(d):
            for j in range(h):
                if schedule[i][j][k]==1:
                    was[j]=1
        res+=4*(was[0]+was[1]+was[2]-1)
    for i in range(d):
        for j in range(h):
            for k1 in range(c):
                for k2 in range(c):
                    if k1!=k2 and tracks[k1]==tracks[k2]:
                        res+=1
    for i in range(d):
        for k in range(c):
            if schedule[i][0][k]+schedule[i][1][k]+schedule[i][2][k]>1:
                res+=700
    for i in range(d):
        for j in range(h):
            for k in range(c):
                if want[i][j][k]==1 and schedule[i][j][k]==0:
                    res+=50
                if dontWant[i][j][k]==1 and schedule[i][j][k]==1:
                    res+=100
    return res+value1(data)*0.005
titles=['Intellectual Property and Technological Innovation',
        'Introduction to Composite Materials and Structures',
        'Thermal Fluid Sciences',
        'Petroleum Geophysics',
        'Introduction Device Physics',
        'Materials Chemistry',
        'Mathematical Methods of Optical Communication',
        'Quantum Fluids',
        'Introduction to Power Systems',
        'Bayesian Methods - Adv Machine Learning',
        'Numerical methods for Partial Differential Equations',
        'Bioinformatics Lab Course 2',
        'RNA Biology',
        'Stem Cells',
        'Space Sector Course',
        'Dynamic Systems and Control'
        ]
titles = ['Intellectual Property and Technological Innovation','Introduction to Composite Materials and Structures',
        'Thermal Fluid Sciences','Petroleum Geophysics','Introduction Device Physics',
        'Materials Chemistry','Mathematical Methods of Optical Communication',
        'Quantum Fluids','Introduction to Power Systems','Bayesian Methods - Adv Machine Learning',
        'Numerical Methods for Partial Differential Equations',
       'Advanced Molecular Biology Techniques 1','Bioinformatics Lab Course 2','RNA Biology','Stem Cells',
        'Space Sector Course','Dynamic Systems and Control']
tracks=['E&I','E&I','Energy','Energy','Energy','Energy','Energy','Energy','Energy','IT','IT','Biomed','Biomed','Biomed','Biomed','Space','Space/IT/PDAM']
dayTitles=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
classes=17
hours=3
days=5
#requiredHours=[3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 3,3 ,3,3,3,3,3]
requiredHours=[3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 3,3, 3, 3, 3, 2, 2]#random numbers
doNotOverlap=np.zeros((classes,classes))
#doNotOverlap[0][1]=1
doNotOverlap[2][8]=1
doNotOverlap[2][2]=1
doNotOverlap[4][6]=1
doNotOverlap[4][7]=1
doNotOverlap[6][7]=1
doNotOverlap[9][10]=1
for i in range(11,14):
    for j in range(11,15):
        if i!=j:
            doNotOverlap[i][j]=1
doNotOverlap[15][16]=1
forbidTime=np.zeros((days,hours,classes))
dontWant=np.zeros((days,hours,classes))
want=np.zeros((days,hours,classes))
for i in range(classes):
    for a in range(hours):
        forbidTime[2][a][i]=1
for d in range(days):
    if d!=2:
        forbidTime[d][0][11]=1
        forbidTime[d][1][11]=1
        forbidTime[d][2][11]=1
    if d!=1 and d!=3:
        forbidTime[d][0][9]=1
        forbidTime[d][1][9]=1
        forbidTime[d][2][9]=1
    forbidTime[d][0][7]=1
    forbidTime[d][2][7]=1
for d in range(days):
    want[d][1][7]=1
    want[d][0][14]=1
want[1][1][1]=1
want[3][1][1]=1
want[4][0][1]=1
want[4][1][1]=1
want[4][2][1]=1
want[1][1][9]=1
want[3][1][9]=1
want[0][1][2]=1
want[4][1][2]=1
want[0][0][10]=1
want[1][0][10]=1
want[3][0][10]=1
forbidTime[2][0][11]=0
forbidTime[2][1][11]=0
forbidTime[2][2][11]=0
for i in range(days):
    forbidTime[i][0][7]=1
    forbidTime[i][2][7]=1
data={'f':value2,'forF':{'classes':classes,'hours':hours,'days':days,'dontWant':dontWant,'want':want,'tracks':tracks,'doNotOverlap':doNotOverlap},'classes':classes,'hours':hours,'days':days,'requiredHours':requiredHours,'doNotOverlap':doNotOverlap,'forbidTime':forbidTime}
an=np.ones(3600)
for i in range(len(an)):
    an[i]=np.power(np.sqrt(3600-i)/60,3)
(value,schedule)=localSearch(data,0,100,an)
print value
for d in range(days):
    print dayTitles[d]
    for i in range(hours):
        res=''
        for k in range(classes):
            if schedule[d][i][k]==1:
                res=res+titles[k]+'('+tracks[k]+'); '
        print i+1,'-',res
    print ''
res={}
for d in range(days):
    today=[]
    for h in range(hours):
        thisHour=[]
        for k in range(classes):
            if schedule[d][h][k]==1:
                thisHour.append(titles[k]+'('+tracks[k]+')')
        today.append(thisHour)
    res[dayTitles[d]]=today
print res

1986.85
Monday
1 - Petroleum Geophysics(Energy); Introduction to Power Systems(Energy); Numerical Methods for Partial Differential Equations(IT); Stem Cells(Biomed); 
2 - Intellectual Property and Technological Innovation(E&I); Thermal Fluid Sciences(Energy); Quantum Fluids(Energy); Bioinformatics Lab Course 2(Biomed); 
3 - Introduction Device Physics(Energy); RNA Biology(Biomed); 

Tuesday
1 - Materials Chemistry(Energy); Mathematical Methods of Optical Communication(Energy); Numerical Methods for Partial Differential Equations(IT); Stem Cells(Biomed); 
2 - Introduction to Composite Materials and Structures(E&I); Quantum Fluids(Energy); Bayesian Methods - Adv Machine Learning(IT); Bioinformatics Lab Course 2(Biomed); 
3 - Introduction Device Physics(Energy); RNA Biology(Biomed); 

Wednesday
1 - Advanced Molecular Biology Techniques 1(Biomed); 
2 - Advanced Molecular Biology Techniques 1(Biomed); 
3 - Advanced Molecular Biology Techniques 1(Biomed); 

Thursday
1 - Materials Chemistry(E

In [12]:
schdl = {'Monday': [], 'Tuesday': [],'Wednesday': [], 'Thursday': [], 'Friday': []}

for i in schdl.keys():
    for j in range(3):
        schdl[i].append([])

schdl['Friday'] = [['Petroleum Geophysics', 'Materials Chemistry', 
                    'Introduction to Power Systems', 'Stem Cells',
                    'Dynamic Systems and Control'], 
                   ['Intellectual Property and Technological Innovation', 
                    'Introduction to Composite Materials and Structures', 
                    'Thermal Fluid Sciences', 'Quantum Fluids', 'Bioinformatics Lab Course 2'], 
                   ['Mathematical Methods of Optical Communication(Energy)', 'Space Sector Course']]

schdl['Tuesday'] =  [['Materials Chemistry', 'Mathematical Methods of Optical Communication',
                      'Numerical Methods for Partial Differential Equations', 'Stem Cells'], 
                     ['Introduction to Composite Materials and Structures', 'Quantum Fluids',
                      'Bayesian Methods - Adv Machine Learning', 'Bioinformatics Lab Course 2'], 
                     ['Introduction Device Physics', 'RNA Biology']]

schdl['Thursday'] = [['Materials Chemistry', 'Introduction to Power Systems', 
                      'Numerical Methods for Partial Differential Equations', 'Dynamic Systems and Control'],
                     ['Intellectual Property and Technological Innovation', 'Introduction to Composite Materials and Structures', 
                      'Introduction Device Physics', 'Bayesian Methods - Adv Machine Learning'], 
                     ['Mathematical Methods of Optical Communication', 'RNA Biology', 
                      'Space Sector Course']]
schdl['Wednesday'] = [['Advanced Molecular Biology Techniques 1'], ['Advanced Molecular Biology Techniques 1'],
                      ['Advanced Molecular Biology Techniques 1']]
schdl['Monday'] =  [['Petroleum Geophysics', 'Introduction to Power Systems', 
                     'Numerical Methods for Partial Differential Equations', 'Stem Cells'], 
                    ['Intellectual Property and Technological Innovation', 'Thermal Fluid Sciences', 
                     'Quantum Fluids', 'Bioinformatics Lab Course 2'],
                    ['Introduction Device Physics', 'RNA Biology']]

In [13]:
# rooms list
# without '450 Space Lab' because we should only assign space to it
rooms = ['401', '402', '403', '407', '408', '421', '423', '441CL', '404 CL']

# dates for stub
dates = ['Feb 1, 8, 15, 22, 29, March 7', 'Feb 2, 9, 16, March 1', 'Feb 3, 10, 17, 24, March 2, 9',
         'Feb 4, 11, 18, 25, March 3, 10', 'Feb 5, 12, 19, 26, March 4, 11']

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
dotw = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

In [14]:
import xlsxwriter
from datetime import datetime, timedelta
import colorsys
from struct import pack

workbook = xlsxwriter.Workbook('schedule_local.xlsx')
ws = workbook.add_worksheet()

np.random.seed(42)
color = np.random.rand()

glob = {
    'bold': 1,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_name': 'Arial',
    'font_size': 10,
    'fg_color': 'white'}

global_format = workbook.add_format(glob)
frm_light = workbook.add_format(glob)
frm_dark = workbook.add_format(glob)
frm_light.set_pattern(1)
frm_dark.set_pattern(1)
frm_light.set_bg_color('#FFFFCC')
frm_dark.set_bg_color('#FFFF99')

for i in range(26):
    for j in range(32):
        ws.write(i, j, None, global_format)

ws.set_row(0, 30)
ws.set_row(1, 30)
ws.set_row(2, 30)
ws.set_row(3, 30)

ws.merge_range('A1:A4', None)

def add_time_col(ws):
    ws.set_column('A:A', 10)
    date_time = datetime.strptime('2016-02-01 09:00:00.000',
                              '%Y-%m-%d %H:%M:%S.%f')
    dt_str = date_time.strftime("%H:%M")    
    
    # 22 time intervals
    for i in xrange(4, 4 + 22):
        ws.set_row(i, 17)
        if (i - 3) == 8:
            next_dt = date_time + timedelta(0,12*60*60+30*60)
        else:
            next_dt = date_time + timedelta(0,30*60)
            
        value = dt_str + ' - ' + next_dt.strftime("%H:%M")
        date_time = next_dt
        dt_str = date_time.strftime("%H:%M")
        
        if ((i - 3) % 7) == 0 and (i-3) < 15:
            ws.write_string(i, 0, value, frm_dark)
        else:
            ws.write_string(i, 0, value, frm_light)
            
add_time_col(ws)
    
#def init_classes():

# http://devmag.org.za/2012/07/29/how-to-choose-colours-procedurally-algorithms/
def gen_color():
    global color
    gr = 0.618033988749895
    color += gr
    color %= 1.
    color_t = (color, 0.5, 0.95)
    rgb_color = colorsys.hsv_to_rgb(*color_t)
    rgb = (np.int(rgb_color[0]*255), np.int(rgb_color[1]*255), np.int(rgb_color[2]*255))
    return '#'+pack("BBB",*rgb).encode('hex')

cls_formats = {}
def get_class_format(cls):
    if not cls in cls_formats:
        class_format = workbook.add_format(glob)
        class_format.set_rotation(90)
        class_format.set_pattern(1)
        class_format.set_bg_color(gen_color())
        class_format.set_text_wrap()
        cls_formats[cls] = class_format
        
    return cls_formats[cls]

def add_day(row_num, col_num, day_name, date_str):
    width = 0
    
    for i in xrange(3):
        width = max(len(schdl[day_name][i]), width)
    
    width = max(2, width)
    
    ws.merge_range(row_num, col_num, row_num, col_num + width - 1, date_str, frm_light)
    row_num += 1
    ws.merge_range(row_num, col_num, row_num, col_num + width - 1, day_name, global_format)
    row_num += 1
    
    for i in xrange(width):
        ws.write_string(row_num, col_num + i, rooms[i], frm_dark)
    
    row_num += 1
    
    for i in xrange(3):
        
        for j, c in enumerate(schdl[day_name][i]):
            ws.merge_range(row_num, col_num + j, row_num + 5, col_num+j, c, get_class_format(c))
        
        it = len(schdl[day_name][i])
        
        for j in xrange(width - it):
            ws.merge_range(row_num, col_num + j + it, row_num + 5, col_num + j + it, None)
        
        ws.merge_range(row_num + 6, col_num, row_num + 6, col_num + width - 1, None, frm_dark)
        
        row_num += 7
        
    return width

i = 0
offset = 1
# adding all the days
for d in dotw:
    if d in schdl:
        offset += add_day(1, offset, d, dates[i])
        i += 1

# header
ws.merge_range(0, 1, 0, offset - 1, "WEEK 1-6", global_format)
    
workbook.close()
